# Explainer notebook

In this notebook I'll explain the steps and thought process behind all my data processing, feature engineering, model creation, validation and optimizations.

The API will consume the outputs of the process described here and will be documented separately.


All the functions i wrote here were later organized in separate modules.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
import seaborn as sns
import kagglehub
import xgboost as xgb

import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

/home/martim/Traive/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Objectives

My chosen task for this challange is to make an Agricultural commodity price forecasting model and implement it.

For this i chose an avocado prices and sales dataset for the US that can be found here: https://www.kaggle.com/datasets/neuromusic/avocado-prices/data

The dataset has weekly entries for Organic and Conventional avocados in lots of regions across the US.

My specific task will be to do a 4 week ahead forecast for the average price of an avocado, for this I'll train one separate regressor model per region.

There are out of the box time series forecasting models, i choose to implement a regression model such as XGB for a few reasons:
1) In my experience if well tuned it tends to perform better.
2) Since this is an exercise that will not be worked upon in the real world and which the objective is purely a  technical evaluation, i believe a custom solution allows me more room to express skills in machine learning engineering than models like prophet.
3) I got some free time this week. In a more time constrained environment where a couple percentage points of precision are not essential I would probably go for the easier route...

## Data understanding

## Loading and preprocessing

In [2]:
def load_raw_data():
    # Download latest version
    path = kagglehub.dataset_download("neuromusic/avocado-prices")

    destination_folder = "./data"
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # Move all files from source to destination
    for filename in os.listdir(path):
        source_file = os.path.join(path, filename)
        destination_file = os.path.join(destination_folder, filename)
        shutil.move(source_file, destination_file)

    df = pd.read_csv('data/avocado.csv')
    return df

df = load_raw_data()

In [3]:
df

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,7,2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,organic,2018,WestTexNewMexico
18245,8,2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,9,2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,10,2018-01-14,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico


The DF has a "Unnamed: 0" column with repeating indices regarding the weeks of the year for each region and each type of avocado. Since i'll be dealing with the dates later this column will be dropped. 

The date column needs to be a datetime, and the names are not standardized, so i'll write a preprocess function to fix all this details}

In [4]:
def preprocess_raw_data(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    # df = df.set_index('Date')
    df = df.drop(['Unnamed: 0', 'year'], axis=1)
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str[0].str.upper() + df.columns.str[1:]
    return df

df = preprocess_raw_data(df)

In [5]:
df

,Date,AveragePrice,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,Type,Region
11569,2015-01-04,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.00,organic,Southeast
9593,2015-01-04,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.00,organic,Chicago
10009,2015-01-04,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.00,organic,HarrisburgScranton
1819,2015-01-04,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.00,conventional,Pittsburgh
9333,2015-01-04,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.00,organic,Boise
...,...,...,...,...,...,...,...,...,...,...,...,...
8574,2018-03-25,1.36,908202.13,142681.06,463136.28,174975.75,127409.04,103579.41,22467.04,1362.59,conventional,Chicago
9018,2018-03-25,0.70,9010588.32,3999735.71,966589.50,30130.82,4014132.29,3398569.92,546409.74,69152.63,conventional,SouthCentral
18141,2018-03-25,1.42,163496.70,29253.30,5080.04,0.00,129163.36,109052.26,20111.10,0.00,organic,SouthCentral
17673,2018-03-25,1.70,190257.38,29644.09,70982.10,0.00,89631.19,89424.11,207.08,0.00,organic,California


## Aggregations

Since i decided that i want to forecast the average avocado prices i need to aggregate the data that currently is separated by types.

In [6]:
# Receives a group and combines all the variables
def aggregate_types(group):
    total_volume = group['TotalVolume'].sum()
    weighted_avg = (group['AveragePrice'] * group['TotalVolume']).sum() / total_volume
    return pd.Series({
        'Date': group['Date'].iloc[0],
        'Region': group['Region'].iloc[0],
        'AveragePrice_combined': weighted_avg, # The average price is wighted against the total volumes
        'TotalVolume_combined': total_volume,
        '4046_combined': group['4046'].sum(),
        '4225_combined': group['4225'].sum(),
        '4770_combined': group['4770'].sum(),
        'TotalBags_combined': group['TotalBags'].sum(),
        'SmallBags_combined': group['SmallBags'].sum(),
        'LargeBags_combined': group['LargeBags'].sum(),
        'XLargeBags_combined': group['XLargeBags'].sum(),
    })

def group_by_region(df):
    combined_df = df.groupby(['Date', 'Region'])[df.columns].apply(aggregate_types).reset_index(drop=True)
    return combined_df

grouped_df = group_by_region(df)

Those are the differences for an example date and region

In [7]:
df[(df['Date'] == '2015-01-04') & (df['Region'] == 'Albany')]

,Date,AveragePrice,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,Type,Region
9177,2015-01-04,1.79,1373.95,57.42,153.88,0.0,1162.65,1162.65,0.00,0.0,organic,Albany
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.9,9716.46,9186.93,529.53,0.0,conventional,Albany


In [8]:
grouped_df[(grouped_df['Date'] == '2015-01-04') & (grouped_df['Region'] == 'Albany')]

,Date,Region,AveragePrice_combined,TotalVolume_combined,4046_combined,4225_combined,4770_combined,TotalBags_combined,SmallBags_combined,LargeBags_combined,XLargeBags_combined
0,2015-01-04,Albany,1.238537,42247.23,2876.92,28441.3,49.9,10879.11,10349.58,529.53,0.0


## Pivoting

If i run the models in this aggregated data i loose the information about the dinamic between organic and conventional sales, so lets get this info back. For this I'll pivot the data and add the values as columns to my aggregated dataset

In [9]:
def pivot_and_merge_numerical_columns(df, grouped_df, target_name):
    numerical_columns = df.select_dtypes(include=['number']).columns
    # numerical_columns = numerical_columns.drop('AveragePrice')
    # Pivots arround date and region to separate each type value a column
    pivot_df = df.pivot(index=['Date','Region'], columns='Type', values=numerical_columns).reset_index()
    # Uses _ to join names
    pivot_df.columns = pivot_df.columns.map(lambda col: '_'.join(map(str, col)).strip('_'))
    # Merges data back toghether with the grouped DF
    merge_df = pd.merge(pivot_df, df[['Date', 'Type', 'Region']], on=['Date', 'Region'], how='left')
    merge_df = pd.merge(merge_df, grouped_df, on=['Date', 'Region'], how='left')
    # Since i pivoted the columns, now each row has the information of the AveragePrice 
    # for both organic and conventional, AND a type flag separating the entries. But since
    # I need a target for the forecast, another merge is done to get the price for the row's type
    merge_df = pd.merge(merge_df, df[['Region', 'Date', 'Type', target_name]], on=['Date', 'Region', 'Type'], how='left')
    return merge_df

target_name = 'AveragePrice'
merge_df = pivot_and_merge_numerical_columns(df, grouped_df, target_name)

In [10]:
merge_df[['Date', 'Region', 'Type' ,'AveragePrice_conventional', 'AveragePrice_organic', 'AveragePrice']].head()

,Date,Region,Type,AveragePrice_conventional,AveragePrice_organic,AveragePrice
0,2015-01-04,Albany,organic,1.22,1.79,1.79
1,2015-01-04,Albany,conventional,1.22,1.79,1.22
2,2015-01-04,Atlanta,conventional,1.00,1.76,1.00
3,2015-01-04,Atlanta,organic,1.00,1.76,1.76
4,2015-01-04,BaltimoreWashington,organic,1.08,1.29,1.29


I have now 2 entries for each date/region pair, one for organic an one for conventional, and those rows have information regarding the other. So i can use this to calculate lagged features in the future and since the grouping is done by date this does not leak future information for the observations

Combining everything in one function we get

In [ ]:
def make_stage_1_data(configs):
    df = load_raw_data()
    df = preprocess_raw_data(df)
    grouped_df = group_by_region(df)
    merge_df = pivot_and_merge_numerical_columns(df, grouped_df, configs['target_name'])
    return merge_df

# Feature engineering

Now i need to make features to feed my regression model.

Since this is an exercise I'll be adding only lags and time related features. In a real world scenario more transformations can be applyied and tested to improve performance, such as seasonal decompositions, rolling statistics, cyclical features and etc...

I need a fuction to select one single region, since I'll be training a separate model for each

In [11]:
def select_region(merge_df, region):
    sel_df = merge_df[merge_df['Region'] == region].reset_index(drop=True)
    return sel_df.drop(columns='Region')

## Lags

This function will create lagged columns for a single region, with an option for adding the region name to the created column (will be usefull later)

In [12]:
def make_lags_single_column(sel_df, lags, lag_column, region_name=False):
    '''Create lag features for a single region and lag column.

    Parameters:
    sel_df (pd.DataFrame): The input DataFrame containing the data.
    lags (list): A list of integers representing the lag periods to create.
    lag_column (str): The name of the column for which to create lag features.
    region_name (str or bool, optional): The name of the region to include in the lag feature names. 
                                         If False, the region name is not included. Default is False.

    Returns:
    pd.DataFrame: A DataFrame containing the lag features.'''
    # TODO Raise error if sel_df has multiple regions
    lag_feats = pd.DataFrame(index=sel_df.index)
    for lag in lags:
        if region_name:
            lag_feats[f'{region_name}_{lag_column}_lag_{lag}'] = sel_df.groupby('Type')[lag_column].shift(lag)
        else:
            lag_feats[f'{lag_column}_lag_{lag}'] = sel_df.groupby('Type')[lag_column].shift(lag)
    return lag_feats

Here is a function that will serve to call the make_lags_single_column() function on desired columns, handling the logic for adding the col name as well.

In [13]:
def make_region_lags(region_filt_df, region_name, columns_to_lag, lags ,region_name_in_col = False):
    lag_features = []
    for col_name in columns_to_lag:
        if region_name_in_col:
            lag_features.append(make_lags_single_column(region_filt_df, lags, col_name, region_name))
        else:
            lag_features.append(make_lags_single_column(region_filt_df, lags, col_name))
    return pd.concat(lag_features, axis=1)

Now i need a way to make the lagged features for a specific region from the merged dataframe.

And while we are at it, could it be that the market behaviour in region A influences the prices in region B?

If we only calculate the lags from the target region, the models will loose this information. 

To account for this, I'll also calculate lags from a set of aggregated regions from the merged dataset and add them to my separate region models. Doing this i can feed the general market information to the models. 

Those "Exogenous" regions will be called aux_regions

To coordinate all those lagging and aux features operations I'll make a configs JSON that will specify what needs to be done

In [14]:
configs = {
        "target_name": "AveragePrice", # Name of the target variable to predict
        "lags":[4], # The lags to be calculated for all the columns of the selected target region
        "aux_regions": ['TotalUS'], # "Exogenous" (auxiliary) regions to be used as inputs when predicting the target region
        "aux_features": ['AveragePrice_combined', 'TotalVolume_combined', # Features of the aux regions to include
                         '4046_combined', '4225_combined', '4770_combined', 
                         'TotalBags_combined', 'SmallBags_combined', 
                         'LargeBags_combined', 'XLargeBags_combined'],
        "aux_lags": [4], # The lags of the aux features to be calculated
    }

# Here lets use only the fourth lag and only one aux_region simplyfy the example

Since the objective is to make a 4 weeks ahead forecast, the minimum lag I can use is 4. Less than that, the model would not work in the real world. 

Eg: if im in week 5 and using the second lag as a feature, my fourth week forecast would need information from week 7, which is in the future.

Now finally i can make the function that will receive the complete merged dataframe, the specified configs and return the lagged columns

In [15]:
def make_target_region_lags_df(merge_df, target_region, train_configs):
    # Lags for the target region
    region_filt_df = select_region(merge_df, target_region)
    columns_to_lag = merge_df.loc[:, merge_df.columns != train_configs['target_name']].select_dtypes(
        include=['number']).columns
    target_region_lags_df = make_region_lags(region_filt_df, target_region, columns_to_lag, train_configs['lags'])

    # Lags for Auxiliary regions
    aux_regions_lags = []
    for aux_region_name in train_configs['aux_regions']:
        if aux_region_name == target_region:
            continue
        
        aux_region_filt_df = select_region(merge_df, aux_region_name).reset_index()
        aux_regions_lags.append(
            make_region_lags(
                aux_region_filt_df, 
                aux_region_name, 
                train_configs['aux_features'], 
                train_configs['aux_lags'],
                region_name_in_col=True))
    aux_regions_lags_df = pd.concat(aux_regions_lags, axis=1)

    return pd.concat([target_region_lags_df, aux_regions_lags_df], axis=1)

Now i can get my lags with

In [16]:
region = 'Albany'
lags_df = make_target_region_lags_df(merge_df, region, configs)

In [17]:
lags_df

,AveragePrice_conventional_lag_4,AveragePrice_organic_lag_4,TotalVolume_conventional_lag_4,TotalVolume_organic_lag_4,4046_conventional_lag_4,4046_organic_lag_4,4225_conventional_lag_4,4225_organic_lag_4,4770_conventional_lag_4,4770_organic_lag_4,...,XLargeBags_combined_lag_4,TotalUS_AveragePrice_combined_lag_4,TotalUS_TotalVolume_combined_lag_4,TotalUS_4046_combined_lag_4,TotalUS_4225_combined_lag_4,TotalUS_4770_combined_lag_4,TotalUS_TotalBags_combined_lag_4,TotalUS_SmallBags_combined_lag_4,TotalUS_LargeBags_combined_lag_4,TotalUS_XLargeBags_combined_lag_4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,1.45,1.43,121804.36,3817.93,8183.48,59.18,95548.47,289.85,61.0,0.0,...,451.11,0.987467,44484806.56,15969142.96,11812643.14,654696.38,16048064.96,11613094.64,4200629.04,234341.28
334,1.43,1.43,85630.24,7566.17,5499.73,4314.30,61661.76,251.85,75.0,0.0,...,380.00,1.100729,38524817.46,13509266.77,11171955.89,554875.19,13288489.86,9806369.66,3254341.43,227778.77
335,1.43,1.43,85630.24,7566.17,5499.73,4314.30,61661.76,251.85,75.0,0.0,...,380.00,1.100729,38524817.46,13509266.77,11171955.89,554875.19,13288489.86,9806369.66,3254341.43,227778.77
336,1.28,1.56,104278.89,5356.63,10368.77,816.56,59723.32,532.59,48.0,0.0,...,310.00,1.077948,41481381.31,13952770.84,10755838.42,725393.48,16046348.64,11431999.60,4310556.28,303792.76


In [18]:
lags_df.columns

Index(['AveragePrice_conventional_lag_4', 'AveragePrice_organic_lag_4',
       'TotalVolume_conventional_lag_4', 'TotalVolume_organic_lag_4',
       '4046_conventional_lag_4', '4046_organic_lag_4',
       '4225_conventional_lag_4', '4225_organic_lag_4',
       '4770_conventional_lag_4', '4770_organic_lag_4',
       'TotalBags_conventional_lag_4', 'TotalBags_organic_lag_4',
       'SmallBags_conventional_lag_4', 'SmallBags_organic_lag_4',
       'LargeBags_conventional_lag_4', 'LargeBags_organic_lag_4',
       'XLargeBags_conventional_lag_4', 'XLargeBags_organic_lag_4',
       'AveragePrice_combined_lag_4', 'TotalVolume_combined_lag_4',
       '4046_combined_lag_4', '4225_combined_lag_4', '4770_combined_lag_4',
       'TotalBags_combined_lag_4', 'SmallBags_combined_lag_4',
       'LargeBags_combined_lag_4', 'XLargeBags_combined_lag_4',
       'TotalUS_AveragePrice_combined_lag_4',
       'TotalUS_TotalVolume_combined_lag_4', 'TotalUS_4046_combined_lag_4',
       'TotalUS_4225_combined_l

## One hot encodes and time features

Add back date, type and target info

In [19]:
region_filt_df = select_region(merge_df, region)
feat_df = pd.concat([
    region_filt_df[['Date', 'Type', configs['target_name']]], 
    lags_df], axis=1)

Regression models need those, so lets add them

In [20]:
def make_time_features(sel_df):
    time_feats = pd.DataFrame(index=sel_df.index)
    dates = sel_df['Date']
    time_feats['Year'] = dates.dt.year
    time_feats['Month'] = dates.dt.month
    time_feats['Day'] = dates.dt.day
    time_feats['DayofWeek'] = dates.dt.dayofweek
    time_feats["WeekofYear"] = dates.dt.isocalendar().week
    time_feats["Quarter"] = dates.dt.quarter
    return time_feats

feat_df = pd.concat([feat_df, make_time_features(feat_df)], axis=1)

One hot encode type

In [21]:
feat_df = pd.get_dummies(feat_df, columns=['Type'], prefix='Type')

Since we are concatenating, with different regions, there is a chance some dates are present in one but not the other, so just dropping the nulls solves it. (In the real world more appropriate data quality checks can be implemented in a prior data engineering stage)

In [22]:
feat_df = feat_df.loc[~feat_df['Date'].isnull(), :]
dates = feat_df['Date'] # also store the dates not to loose the info (usefull for plotting)
feat_df = feat_df.drop(columns='Date')

Finally, now I'll make the model ready data

In [23]:
X = feat_df[feat_df.columns.difference([configs['target_name']])]
y = feat_df[configs['target_name']]

In [24]:
X

,4046_combined_lag_4,4046_conventional_lag_4,4046_organic_lag_4,4225_combined_lag_4,4225_conventional_lag_4,4225_organic_lag_4,4770_combined_lag_4,4770_conventional_lag_4,4770_organic_lag_4,AveragePrice_combined_lag_4,...,TotalVolume_combined_lag_4,TotalVolume_conventional_lag_4,TotalVolume_organic_lag_4,Type_conventional,Type_organic,WeekofYear,XLargeBags_combined_lag_4,XLargeBags_conventional_lag_4,XLargeBags_organic_lag_4,Year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,True,1,NaN,NaN,NaN,2015
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,False,1,NaN,NaN,NaN,2015
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,True,2,NaN,NaN,NaN,2015
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,False,2,NaN,NaN,NaN,2015
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,False,3,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,8242.66,8183.48,59.18,95838.32,95548.47,289.85,61.0,61.0,0.0,1.449392,...,125622.29,121804.36,3817.93,False,True,10,451.11,451.11,0.0,2018
334,9814.03,5499.73,4314.30,61913.61,61661.76,251.85,75.0,75.0,0.0,1.430000,...,93196.41,85630.24,7566.17,False,True,11,380.00,380.00,0.0,2018
335,9814.03,5499.73,4314.30,61913.61,61661.76,251.85,75.0,75.0,0.0,1.430000,...,93196.41,85630.24,7566.17,True,False,11,380.00,380.00,0.0,2018
336,11185.33,10368.77,816.56,60255.91,59723.32,532.59,48.0,48.0,0.0,1.293680,...,109635.52,104278.89,5356.63,True,False,12,310.00,310.00,0.0,2018


In [25]:
X.columns

Index(['4046_combined_lag_4', '4046_conventional_lag_4', '4046_organic_lag_4',
       '4225_combined_lag_4', '4225_conventional_lag_4', '4225_organic_lag_4',
       '4770_combined_lag_4', '4770_conventional_lag_4', '4770_organic_lag_4',
       'AveragePrice_combined_lag_4', 'AveragePrice_conventional_lag_4',
       'AveragePrice_organic_lag_4', 'Day', 'DayofWeek',
       'LargeBags_combined_lag_4', 'LargeBags_conventional_lag_4',
       'LargeBags_organic_lag_4', 'Month', 'Quarter',
       'SmallBags_combined_lag_4', 'SmallBags_conventional_lag_4',
       'SmallBags_organic_lag_4', 'TotalBags_combined_lag_4',
       'TotalBags_conventional_lag_4', 'TotalBags_organic_lag_4',
       'TotalUS_4046_combined_lag_4', 'TotalUS_4225_combined_lag_4',
       'TotalUS_4770_combined_lag_4', 'TotalUS_AveragePrice_combined_lag_4',
       'TotalUS_LargeBags_combined_lag_4', 'TotalUS_SmallBags_combined_lag_4',
       'TotalUS_TotalBags_combined_lag_4',
       'TotalUS_TotalVolume_combined_lag_4',
    

In [26]:
y

0      1.79
1      1.22
2      1.77
3      1.24
4      1.17
       ... 
333    1.68
334    1.66
335    1.35
336    1.57
337    1.71
Name: AveragePrice, Length: 338, dtype: float64

Combining all the feat eng in one function we get

In [ ]:
def make_stage_2_data(merge_df, region, train_configs):
    lags_df = make_target_region_lags_df(merge_df, region, train_configs)
    region_filt_df = select_region(merge_df, region)
    feat_df = pd.concat([
        region_filt_df[['Date', 'Type', train_configs['target_name']]], 
        lags_df], axis=1)

    feat_df = pd.concat([feat_df, make_time_features(feat_df)], axis=1)
    feat_df = pd.get_dummies(feat_df, columns=['Type'], prefix='Type')

    feat_df = feat_df.loc[~feat_df['Date'].isnull(), :]
    dates = feat_df['Date']
    feat_df = feat_df.drop(columns='Date')
    
    X = feat_df[feat_df.columns.difference([train_configs['target_name']])]
    y = feat_df[train_configs['target_name']]
    
    return X, y, dates

# Modelling and validation

## Hyperparameter optimization

Before the main train loop i made a grid search with optuna to optimize the models. The scores and parameters were logged to MLflow

To specify which regions to iterate I'll add a key "target_regions" to the configs like this

In [ ]:
# configs = {
#         "target_name": "AveragePrice", # Name of the target variable to predict
#         "lags":[4, 8, 13, 26, 52],
#         'target_regions': merge_df['Region'].unique(),
#         "aux_regions": ['TotalUS', 'West', 'Midsouth', 'Northeast', 'Southeast', 'SouthCentral'], # "Exogenous" (auxiliary) regions to be used as inputs when predicting the target region
#         "aux_features": ['AveragePrice_combined', 'TotalVolume_combined', # Features of the aux regions to include
#                          '4046_combined', '4225_combined', '4770_combined', 
#                          'TotalBags_combined', 'SmallBags_combined', 
#                          'LargeBags_combined', 'XLargeBags_combined'],
#         "aux_lags": [4, 8, 13, 26, 52],
#     }

This was the code executed to get the best parameters for each region, but since it would take too long to run it here, ill be skipping it. A JSON file with my results is included in the repo.

In [ ]:
# import mlflow
# import mlflow.xgboost
# import xgboost as xgb
# from sklearn.model_selection import TimeSeriesSplit, train_test_split
# from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
# import numpy as np
# import pandas as pd
# import os
# import optuna
# import feature_eng

# # Set your MLflow experiment name
# mlflow.set_experiment("Optuna Hyperparameter Optimization")

# # Define the objective function for Optuna
# def objective(trial):
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 50, 200),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#         "max_depth": trial.suggest_int("max_depth", 3, 7),
#         "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
#         "gamma": trial.suggest_float("gamma", 0, 5),
#         "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
#         "reg_lambda": trial.suggest_float("reg_lambda", 0, 5)
#     }
    
#     fold_mse = []
#     fold_mape = []
    
#     for fold, (train_cv_index, val_cv_index) in enumerate(tscv.split(X_train)):
#         X_train_cv, X_val_cv = X_train.iloc[train_cv_index], X_train.iloc[val_cv_index]
#         y_train_cv, y_val_cv = y_train.iloc[train_cv_index], y_train.iloc[val_cv_index]
        
#         model = xgb.XGBRegressor(**params)
#         model.fit(X_train_cv, y_train_cv)
        
#         # Make predictions and compute performance
#         y_pred_cv = model.predict(X_val_cv)
#         mse = mean_squared_error(y_val_cv, y_pred_cv)
#         mape = mean_absolute_percentage_error(y_val_cv, y_pred_cv)
#         fold_mse.append(mse)
#         fold_mape.append(mape)
    
#     avg_mse = np.mean(fold_mse)   
#     avg_mape = np.mean(fold_mape)    
    
#     return avg_mape

# for region in configs['target_regions']:
#     print(f'Training: {region}')
#     X, y, dates = feature_eng.make_stage_2_data(merge_df, region, configs)

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
#     # Start one run per region
#     with mlflow.start_run(run_name=f"Region: {region}") as region_run:
#         mlflow.log_param("region", region)
        
#         # Expanding window CV.
#         tscv = TimeSeriesSplit(n_splits=5)
        
#         # Perform hyperparameter optimization with Optuna
#         study = optuna.create_study(direction="minimize")
#         study.optimize(objective, n_trials=40)
        
#         # Log the best parameters
#         best_params = study.best_params
#         mlflow.log_params(best_params)
        
#         # Log the best cross-validation scores
#         # best_mse = study.best_value
#         best_mape = study.best_value
#         mlflow.log_metric("best_cv_mape", best_mape)

## Main tran loop

The model is trained on the entire train set using the best cross validated parameters for each region. After that there is another retraining, this time with the entire dataset, the retrained model is then logged and registered to an MLflow server to be later fetched for the deploy.

Lets define a smaller set of target regions so it would not take long

In [ ]:
configs = {
        "target_name": "AveragePrice", # Name of the target variable to predict
        "lags":[4], # The lags to be calculated for all the columns of the selected target region
        'target_regions':['Chicago', 'Albany'], # The regions i want to forecast
        "aux_regions": ['TotalUS', 'West', 'Midsouth', 'Northeast', 'Southeast', 'SouthCentral'], # "Exogenous" (auxiliary) regions to be used as inputs when predicting the target region
        "aux_features": ['AveragePrice_combined', 'TotalVolume_combined', # Features of the aux regions to include
                         '4046_combined', '4225_combined', '4770_combined', 
                         'TotalBags_combined', 'SmallBags_combined', 
                         'LargeBags_combined', 'XLargeBags_combined'],
        "aux_lags": [4], # The lags to be calculated of the aux features
    }

In [ ]:
import json

def load_best_params(region, from_json=True):
    if from_json:
        with open('best_params.json') as file:
            best_params = json.load(file)
        return best_params[region]
    
    # If not loading with a JSON, load with MLflow TODO
    return

For visualizing results easily I'll make a plotting function

In [ ]:
def plot_results(y_train, y_true, y_pred, target_name, dates, fold=None):
    # Plot the fold results
    plt.figure(figsize=(12, 6))
    plt.plot(dates.loc[y_train.index], y_train, label='Train')
    plt.plot(dates.loc[y_true.index], y_true, label='True')
    plt.plot(dates.loc[y_true.index], y_pred, label='Predicted')
    plt.legend()
    if fold:
        plt.title(f"Train, Validation, and Predicted Values - Fold {fold}")
    else:
        plt.title(f"Train, Test, and Predicted Values")
    plt.xlabel("Date")
    plt.ylabel(target_name)
    return plt


Now, the main train loop

In [ ]:
mlflow.set_experiment("Example Experiment")

for region in configs['target_regions']:
    print(f'Trainning: {region}')
    X, y, dates = make_stage_2_data(merge_df, region, configs)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Start one run per region
    with mlflow.start_run(run_name=f"Region: {region}") as region_run:
        mlflow.log_param("region", region)

        # get best params from grid search
        params = load_best_params(region, from_json=True)

        # Evaluate final model on the hold-out test set using training data only
        final_model_cv = xgb.XGBRegressor(**params)
        final_model_cv.fit(X_train, y_train)
        y_test_pred = final_model_cv.predict(X_test)
        test_mse = mean_squared_error(y_test, y_test_pred)
        mlflow.log_metric("test_mse", test_mse)
        
        plot = plot_results(y_train, y_test, y_test_pred, configs['target_name'], dates)
        plot_path = f"plot_test.png"
        plot.savefig(plot_path)
        mlflow.log_artifact(plot_path)
        plot.close()
        # Delete the plot file after logging it
        if os.path.exists(plot_path):
            os.remove(plot_path)
        
        final_model = xgb.XGBRegressor(**params)
        final_model.fit(X, y)

        mlflow.xgboost.log_model(final_model, artifact_path="final_model", input_example=X.iloc[:1])
        
        # Register the model
        model_uri = mlflow.get_artifact_uri("final_model")
        mlflow.register_model(model_uri, name=f'{region}_AVOCADO_FORECAST_EXAMPLE')


2025/03/10 19:40:14 INFO mlflow.tracking.fluent: Experiment with name 'Example Experiment' does not exist. Creating a new experiment.


Trainning: Chicago


/home/martim/Traive/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/home/martim/Traive/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:40:23] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj`

Trainning: SanFrancisco


/home/martim/Traive/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/home/martim/Traive/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:40:36] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj`

The models can now be analyzed inside MLFlow and deployed for serving predictions with a restAPI that loads them from the MLFlow model registry

# Runned Grid Search (save results to json then delete later)

In [58]:
configs = {
        "target_name": "AveragePrice", # Name of the target variable to predict
        # "lags":[4], # The lags to be calculated for all the columns of the selected target region
        "lags":[4, 8, 13, 26, 52],
        'target_regions':['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
            'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
            'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
            'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
            'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
            'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
            'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
            'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
            'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
            'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
            'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
            'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
            'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'],
        "aux_regions": ['TotalUS', 'West', 'Midsouth', 'Northeast', 'Southeast', 'SouthCentral'], # "Exogenous" (auxiliary) regions to be used as inputs when predicting the target region
        "aux_features": ['AveragePrice_combined', 'TotalVolume_combined', # Features of the aux regions to include
                         '4046_combined', '4225_combined', '4770_combined', 
                         'TotalBags_combined', 'SmallBags_combined', 
                         'LargeBags_combined', 'XLargeBags_combined'],
        # "aux_lags": [4], # The lags to be calculated of the aux features
        "aux_lags": [4, 8, 13, 26, 52],
    }

In [59]:
import data_processing
merge_df = data_processing.make_stage_1_data(configs)

In [ ]:
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np
import pandas as pd
import os
import optuna
import feature_eng

# Set your MLflow experiment name
mlflow.set_experiment("Optuna Hyperparameter Optimization")

# Define the objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 5)
    }
    
    fold_mse = []
    fold_mape = []
    
    for fold, (train_cv_index, val_cv_index) in enumerate(tscv.split(X_train)):
        X_train_cv, X_val_cv = X_train.iloc[train_cv_index], X_train.iloc[val_cv_index]
        y_train_cv, y_val_cv = y_train.iloc[train_cv_index], y_train.iloc[val_cv_index]
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train_cv, y_train_cv)
        
        # Make predictions and compute performance
        y_pred_cv = model.predict(X_val_cv)
        mse = mean_squared_error(y_val_cv, y_pred_cv)
        mape = mean_absolute_percentage_error(y_val_cv, y_pred_cv)
        fold_mse.append(mse)
        fold_mape.append(mape)
    
    avg_mse = np.mean(fold_mse)   
    avg_mape = np.mean(fold_mape)    
    
    return avg_mape

for region in configs['target_regions']:
    print(f'Training: {region}')
    X, y, dates = feature_eng.make_stage_2_data(merge_df, region, configs)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Start one run per region
    with mlflow.start_run(run_name=f"Region: {region}") as region_run:
        mlflow.log_param("region", region)
        
        # Expanding window CV.
        tscv = TimeSeriesSplit(n_splits=5)
        
        # Perform hyperparameter optimization with Optuna
        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=40)
        
        # Log the best parameters
        best_params = study.best_params
        mlflow.log_params(best_params)
        
        # Log the best cross-validation scores
        # best_mse = study.best_value
        best_mape = study.best_value
        mlflow.log_metric("best_cv_mape", best_mape)

2025/03/10 21:27:46 INFO mlflow.tracking.fluent: Experiment with name 'Optuna Hyperparameter Optimization' does not exist. Creating a new experiment.
[I 2025-03-10 21:27:46,544] A new study created in memory with name: no-name-101772f5-695b-49da-a824-1f21f7b7a405


Training: Albany


[I 2025-03-10 21:27:50,444] Trial 0 finished with value: 0.14486099069523398 and parameters: {'n_estimators': 50, 'learning_rate': 0.14525944963878198, 'max_depth': 7, 'subsample': 0.7904456328795267, 'colsample_bytree': 0.9021469732313794, 'gamma': 4.6533830625244645, 'reg_alpha': 3.4076139356519146, 'reg_lambda': 0.40106622438011386}. Best is trial 0 with value: 0.14486099069523398.
[I 2025-03-10 21:27:58,308] Trial 1 finished with value: 0.11986225591163772 and parameters: {'n_estimators': 198, 'learning_rate': 0.10942029793843307, 'max_depth': 4, 'subsample': 0.5897701155520684, 'colsample_bytree': 0.5482939691496018, 'gamma': 2.4526579382419027, 'reg_alpha': 0.038022844495874675, 'reg_lambda': 2.7943765782226166}. Best is trial 1 with value: 0.11986225591163772.
[I 2025-03-10 21:28:02,733] Trial 2 finished with value: 0.1401484685146662 and parameters: {'n_estimators': 70, 'learning_rate': 0.03088143368126505, 'max_depth': 4, 'subsample': 0.7102987585250911, 'colsample_bytree': 0.

Training: Atlanta


[I 2025-03-10 21:32:23,002] Trial 0 finished with value: 0.24413914909925225 and parameters: {'n_estimators': 177, 'learning_rate': 0.18027783017464025, 'max_depth': 4, 'subsample': 0.5565433059109752, 'colsample_bytree': 0.670795840654648, 'gamma': 2.7819728410026947, 'reg_alpha': 3.6902327271857676, 'reg_lambda': 0.9475265657561455}. Best is trial 0 with value: 0.24413914909925225.
[I 2025-03-10 21:32:29,864] Trial 1 finished with value: 0.23115215751193857 and parameters: {'n_estimators': 158, 'learning_rate': 0.015380016299151492, 'max_depth': 7, 'subsample': 0.7128411755994215, 'colsample_bytree': 0.598402013032534, 'gamma': 2.8959657783596384, 'reg_alpha': 0.9485373859091634, 'reg_lambda': 2.7232787498995137}. Best is trial 1 with value: 0.23115215751193857.
[I 2025-03-10 21:32:35,817] Trial 2 finished with value: 0.2351774430490508 and parameters: {'n_estimators': 105, 'learning_rate': 0.17436448820492065, 'max_depth': 5, 'subsample': 0.9466837841736462, 'colsample_bytree': 0.74

...

[I 2025-03-10 23:11:07,030] Trial 0 finished with value: 0.17463404285548365 and parameters: {'n_estimators': 77, 'learning_rate': 0.030822824355666814, 'max_depth': 4, 'subsample': 0.797188555291136, 'colsample_bytree': 0.9800195112415357, 'gamma': 1.9017638614617587, 'reg_alpha': 1.875211514276887, 'reg_lambda': 1.9387821604360234}. Best is trial 0 with value: 0.17463404285548365.
[I 2025-03-10 23:11:10,157] Trial 1 finished with value: 0.18443115706247365 and parameters: {'n_estimators': 91, 'learning_rate': 0.18099352392753934, 'max_depth': 4, 'subsample': 0.5667614399766487, 'colsample_bytree': 0.5985648156082051, 'gamma': 3.3255377821318355, 'reg_alpha': 3.377161949411111, 'reg_lambda': 4.704097695241703}. Best is trial 0 with value: 0.17463404285548365.
[I 2025-03-10 23:11:12,686] Trial 2 finished with value: 0.1843400043624948 and parameters: {'n_estimators': 98, 'learning_rate': 0.10040892901701887, 'max_depth': 4, 'subsample': 0.9132766463444164, 'colsample_bytree': 0.8313127

Training: Midsouth


[I 2025-03-10 23:13:57,366] Trial 0 finished with value: 0.15691366452486671 and parameters: {'n_estimators': 126, 'learning_rate': 0.15855856996929038, 'max_depth': 6, 'subsample': 0.7237754885978522, 'colsample_bytree': 0.6403310252439072, 'gamma': 2.6575942443188, 'reg_alpha': 4.149621069674236, 'reg_lambda': 3.1036674176965446}. Best is trial 0 with value: 0.15691366452486671.
[I 2025-03-10 23:14:01,799] Trial 1 finished with value: 0.14913806746991595 and parameters: {'n_estimators': 157, 'learning_rate': 0.029726338682819785, 'max_depth': 3, 'subsample': 0.5037909048227179, 'colsample_bytree': 0.8855589149463283, 'gamma': 2.862544540962115, 'reg_alpha': 0.11967835624760725, 'reg_lambda': 3.673802343205981}. Best is trial 1 with value: 0.14913806746991595.
[I 2025-03-10 23:14:05,135] Trial 2 finished with value: 0.11904480723221525 and parameters: {'n_estimators': 148, 'learning_rate': 0.05935399828438398, 'max_depth': 3, 'subsample': 0.7301883343665354, 'colsample_bytree': 0.7510

Training: Nashville


[I 2025-03-10 23:16:34,070] Trial 0 finished with value: 0.2385907943116988 and parameters: {'n_estimators': 68, 'learning_rate': 0.04568718761276516, 'max_depth': 3, 'subsample': 0.5593544630139063, 'colsample_bytree': 0.8067688692821245, 'gamma': 2.6467322190066422, 'reg_alpha': 0.11033895053060405, 'reg_lambda': 2.937158806337263}. Best is trial 0 with value: 0.2385907943116988.
[I 2025-03-10 23:16:38,226] Trial 1 finished with value: 0.24674451511531625 and parameters: {'n_estimators': 132, 'learning_rate': 0.12986480904386277, 'max_depth': 7, 'subsample': 0.5628276292090246, 'colsample_bytree': 0.9113967781389687, 'gamma': 3.198926667823818, 'reg_alpha': 0.360031467858668, 'reg_lambda': 3.5423792193934824}. Best is trial 0 with value: 0.2385907943116988.
[I 2025-03-10 23:16:41,597] Trial 2 finished with value: 0.24379906593779346 and parameters: {'n_estimators': 89, 'learning_rate': 0.0367310077445014, 'max_depth': 4, 'subsample': 0.7110420544205229, 'colsample_bytree': 0.85782664

# Other section